In [3]:
import datetime as dt
import sys
sys.path.append('../../')
from prospector.core.extractTwitter import ExtractFromTwitter
from prospector.core.functions import get_current_date

INFO: {'User-Agent': 'Mozilla/5.0 (compatible, MSIE 11, Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko'}


# extractng information from twitter

In [4]:
begin_date = dt.date(2020, 1, 1)
end_date = get_current_date()

In [5]:
#https://lifehacker.com/search-twitter-more-efficiently-with-these-search-opera-1598165519

In [6]:
sam = ExtractFromTwitter(begin_date,end_date)
sam.search_terms = "COVID-19"
sam.company_tweets_to_dictionary()

INFO: queries: ['bridgstone AND COVID-19 since:2020-01-01 until:2020-01-07', 'bridgstone AND COVID-19 since:2020-01-07 until:2020-01-13', 'bridgstone AND COVID-19 since:2020-01-13 until:2020-01-20', 'bridgstone AND COVID-19 since:2020-01-20 until:2020-01-26', 'bridgstone AND COVID-19 since:2020-01-26 until:2020-02-01', 'bridgstone AND COVID-19 since:2020-02-01 until:2020-02-08', 'bridgstone AND COVID-19 since:2020-02-08 until:2020-02-14', 'bridgstone AND COVID-19 since:2020-02-14 until:2020-02-20', 'bridgstone AND COVID-19 since:2020-02-20 until:2020-02-27', 'bridgstone AND COVID-19 since:2020-02-27 until:2020-03-04', 'bridgstone AND COVID-19 since:2020-03-04 until:2020-03-10', 'bridgstone AND COVID-19 since:2020-03-10 until:2020-03-17', 'bridgstone AND COVID-19 since:2020-03-17 until:2020-03-23', 'bridgstone AND COVID-19 since:2020-03-23 until:2020-03-29', 'bridgstone AND COVID-19 since:2020-03-29 until:2020-04-05', 'bridgstone AND COVID-19 since:2020-04-05 until:2020-04-11', 'bridgst

INFO: Got 2 tweets (2 new).
INFO: Got 10 tweets (8 new).
INFO: Got 34 tweets (24 new).
INFO: Got 62 tweets (28 new).
INFO: Got 74 tweets (12 new).
INFO: Got 84 tweets (10 new).
INFO: Got 118 tweets (34 new).
INFO: Got 176 tweets (58 new).
INFO: Got 228 tweets (52 new).
INFO: Got 270 tweets (42 new).
INFO: Got 317 tweets (47 new).
INFO: Got 377 tweets (60 new).
INFO: Got 377 tweets (0 new).
INFO: Got 377 tweets (0 new).
INFO: Got 377 tweets (0 new).
INFO: Got 377 tweets (0 new).
INFO: Got 377 tweets (0 new).
INFO: Got 377 tweets (0 new).
INFO: Got 377 tweets (0 new).
INFO: Got 377 tweets (0 new).
INFO: queries: ['goodyear AND COVID-19 since:2020-01-01 until:2020-01-07', 'goodyear AND COVID-19 since:2020-01-07 until:2020-01-13', 'goodyear AND COVID-19 since:2020-01-13 until:2020-01-20', 'goodyear AND COVID-19 since:2020-01-20 until:2020-01-26', 'goodyear AND COVID-19 since:2020-01-26 until:2020-02-01', 'goodyear AND COVID-19 since:2020-02-01 until:2020-02-08', 'goodyear AND COVID-19 sinc

In [7]:
sam.company_tweets.keys()

dict_keys(['bridgstone', 'cooper', 'firestone', 'michelin', 'pirelli', 'goodyear', 'dunlop', 'bfgoodrich'])

In [8]:
sam.company_tweets['cooper'].timestamp[0], sam.company_tweets['cooper'].text[0]

(Timestamp('2020-04-03 22:14:40'),
 'Cooper Tire plants to remain idled as COVID-19 spreads http://www.rubbernews.com/coronavirus/cooper-tire-plants-remain-idled-covid-19-spreads\xa0… via @rubbernews')

In [9]:
sam.compile_company_tweet_dictionary_to_pandas()

In [10]:
df_covid_tweets = sam.df_tweets.reset_index(drop=True)

In [65]:
df_covid_tweets.columns.tolist()

['company',
 'screen_name',
 'username',
 'user_id',
 'tweet_id',
 'tweet_url',
 'timestamp',
 'timestamp_epochs',
 'text',
 'text_html',
 'links',
 'hashtags',
 'has_media',
 'img_urls',
 'video_url',
 'likes',
 'retweets',
 'replies',
 'is_replied',
 'is_reply_to',
 'parent_tweet_id',
 'reply_to_users']

In [72]:
df_covid_tweets[['timestamp', 'text']]

,timestamp,text
0,2020-05-06 21:05:14,Michelin-starred San Francisco restaurant @kin...
1,2020-05-06 21:05:14,Michelin-starred San Francisco restaurant @kin...
2,2020-05-06 20:37:09,Happy #NationalNursesDay! The Goodyear Police ...
3,2020-05-06 20:37:09,Happy #NationalNursesDay! The Goodyear Police ...
4,2020-05-06 19:18:21,@NumisSecurities Industrial Property Conferenc...
...,...,...
2343,2020-02-29 12:01:27,#Michelin hat die #Sterneverleihung des #MICHE...
2344,2020-02-25 16:36:39,Coronavirus updates: 5 dead and 200 infected i...
2345,2020-02-25 16:36:39,Coronavirus updates: 5 dead and 200 infected i...
2346,2020-02-19 11:19:27,PIRELLI OPERATIONS SUSPENDED IN 2 CHINESE FACT...


# NLP

In [12]:
import spacy
nlp = spacy.load("en_core_web_md")

In [13]:
from spacy.lang.en.stop_words import STOP_WORDS
import scattertext
import pandas as pd
from spacy.matcher import PhraseMatcher

In [14]:
doc = df_covid_tweets.text.apply(lambda x: nlp(x.lower()))

In [46]:
matcher = PhraseMatcher(nlp.vocab)
pattern1 = nlp('factories')
pattern2 = nlp('furloughing')
pattern3 = nlp('coronavirus pandemic')
matcher.add('PLACE', None, pattern1)
matcher.add('EVENT', None, pattern2)
matcher.add('EVENT', None, pattern3)

In [47]:
#find the match, ID tweet
for i, line in enumerate(doc):
    for match_id, start, end in matcher(line):
        span = line[start:end]
        print('macthed span tweet# ' + str(i) +':', span.text)

macthed span tweet# 129: factories
macthed span tweet# 130: factories
macthed span tweet# 585: coronavirus pandemic
macthed span tweet# 586: coronavirus pandemic
macthed span tweet# 605: coronavirus pandemic
macthed span tweet# 606: coronavirus pandemic
macthed span tweet# 1137: coronavirus pandemic
macthed span tweet# 1138: coronavirus pandemic
macthed span tweet# 1268: furloughing
macthed span tweet# 1268: coronavirus pandemic
macthed span tweet# 1269: furloughing
macthed span tweet# 1269: coronavirus pandemic
macthed span tweet# 1279: furloughing
macthed span tweet# 1279: coronavirus pandemic
macthed span tweet# 1280: furloughing
macthed span tweet# 1280: coronavirus pandemic
macthed span tweet# 1281: furloughing
macthed span tweet# 1281: coronavirus pandemic
macthed span tweet# 1282: furloughing
macthed span tweet# 1282: coronavirus pandemic
macthed span tweet# 1283: furloughing
macthed span tweet# 1283: coronavirus pandemic
macthed span tweet# 1284: furloughing
macthed span tweet#

In [61]:
#look at tweet with matched phrase
focused_tweet = doc[1137]
print(focused_tweet)

bridgestone retail operations is a proud supporter of @bgca_clubs whose services are even more important now during the coronavirus pandemic. to support, bsro is giving all customer donations made in q1 2020 to the bgca covid-19 relief fund. http://ow.ly/rl5f50z6qfh pic.twitter.com/bbrgyxtiuw


In [54]:
for token in focused_tweet:
    print('{:<12}{:<10}{:<10}'.format(token.text, token.pos_, token.dep_))

bridgestone PROPN     compound  
retail      ADJ       amod      
operations  NOUN      nsubj     
is          AUX       ROOT      
a           DET       det       
proud       ADJ       amod      
supporter   NOUN      attr      
of          ADP       prep      
@bgca_clubs PROPN     pobj      
whose       DET       poss      
services    NOUN      nsubj     
are         AUX       relcl     
even        ADV       advmod    
more        ADV       advmod    
important   ADJ       acomp     
now         ADV       advmod    
during      ADP       prep      
the         DET       det       
coronavirus NOUN      compound  
pandemic    NOUN      pobj      
.           PUNCT     punct     
to          PART      aux       
support     VERB      advcl     
,           PUNCT     punct     
bsro        PROPN     nsubj     
is          AUX       aux       
giving      VERB      ROOT      
all         DET       det       
customer    NOUN      compound  
donations   NOUN      dobj      
made      

In [56]:
# test.to_json()
# spacy.explain('PROPN')
hash_ = nlp.vocab.strings[str(focused_tweet[0])]
print(hash_)
print(nlp.vocab.strings[hash_])

17434099631913130510
bridgestone


In [57]:
[noun for noun in focused_tweet.noun_chunks]

[bridgestone retail operations,
 a proud supporter,
 @bgca_clubs,
 whose services,
 the coronavirus pandemic,
 bsro,
 all customer donations,
 q1,
 the bgca covid-19 relief fund]

In [59]:
sentences = [sent for sent in focused_tweet.sents]
print(sentences)

[bridgestone retail operations is a proud supporter of @bgca_clubs whose services are even more important now during the coronavirus pandemic., to support, bsro is giving all customer donations made in q1 2020 to the bgca covid-19 relief fund., http://ow.ly/rl5f50z6qfh pic.twitter.com/bbrgyxtiuw]
